In [3]:
import pandas as pd
import re
from collections import OrderedDict

In [4]:
with open('../artforum index v3.txt') as f:
    catalog = f.read()
    
catalog = catalog.replace("NOTE: Please read issue and page references as in the following example:", "") #removing troublesome note lines
catalog = catalog.replace("VI:8, 40-5 April 1968 = Volume VI (6), number 8, pages 40 through 45 April 1968", "")

In [5]:
split_on_index = re.split(r'^([A-Z]{1})$', catalog, flags=re.MULTILINE)

In [6]:
intro = split_on_index[0]
del split_on_index[0]
all_entries = OrderedDict(zip(split_on_index[0::2], split_on_index[1::2]))
print (all_entries.keys())

odict_keys(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z'])


In [8]:
writer = pd.ExcelWriter('catalog_tabulated.xlsx', engine='xlsxwriter')

for index_letter, content in all_entries.items():
    regex_str = r'^({}[A-Z]*\b.*)$'.format(index_letter)
    entries = re.split(regex_str, content, flags=re.MULTILINE)
    title_data_list = []
    del entries[0]  ## first string returned by split is notes and blank lines
    title = None
    data = None
    for entry in entries:
        if re.match(regex_str, entry):
            title_data_list.append((title, data))
            title = entry.strip()
            data = None
        else:
            data = entry.strip()
    
    if title_data_list[0][0] is None and title_data_list[0][1] is None:
        del title_data_list[0]
    
    df = pd.DataFrame(title_data_list)
    df.columns = ['entry_title', 'entry_data']
    
    df.to_excel(writer, sheet_name=index_letter)
    
    
writer.save()

In [89]:
#todo

def parse_entry_title(title_string):
    pass

def parse_entry_data(data_string):
    pass